<a href="https://colab.research.google.com/github/101Vaibhav04/Portfolio-Optimization/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt


def calculate_weekly_returns(df):
    weekly_data = df.resample('W').last()
    weekly_returns = weekly_data.pct_change().dropna()
    return weekly_returns


def rank_stocks(weekly_returns, window=50):
    avg_returns = weekly_returns.rolling(window=window).mean()
    latest_avg_returns = avg_returns.iloc[-1].sort_values(ascending=False)
    return latest_avg_returns.index.tolist()

def get_top_stocks(ranked_stocks, n=20):
    return ranked_stocks[:n]

def exponentially_weighted_cov_matrix(returns, alpha=0.94):
    n = len(returns)
    weights = np.array([(1 - alpha) ** i for i in range(n)][::-1])
    weights /= weights.sum()
    weighted_cov_matrix = np.cov(returns, rowvar=False, aweights=weights)
    return weighted_cov_matrix

def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def create_min_variance_portfolio(returns, alpha=0.94):
    n = len(returns.columns)
    init_guess = np.array([1/n for _ in range(n)])
    bounds = tuple((0, 1) for _ in range(n))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})


    cov_matrix = exponentially_weighted_cov_matrix(returns.values, alpha)


    result = minimize(portfolio_variance, init_guess, args=(cov_matrix,),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x


def get_stock_weightage(weekly_returns, current_date, alpha=0.94):
    data_slice = weekly_returns.loc[:current_date].iloc[-50:]

    if len(data_slice) < 50:
        raise ValueError("Not enough historical data before the target date.")

    ranked_stocks = rank_stocks(data_slice)
    top_stocks = get_top_stocks(ranked_stocks)
    top_stock_returns = data_slice[top_stocks]


    weights = create_min_variance_portfolio(top_stock_returns, alpha)
    stock_weights = dict(zip(top_stocks, weights))

    return stock_weights

def calculate_transaction_costs(prev_weights, new_weights, transaction_cost=0.001):
    weight_diff = np.abs(np.array(new_weights) - np.array(prev_weights))
    total_transaction_cost = np.sum(weight_diff) * transaction_cost
    return total_transaction_cost


def backtest_strategy(df, transaction_cost=0.001, alpha=0.94):
    weekly_returns = calculate_weekly_returns(df)
    portfolio_returns = []
    weightage_history = []

    prev_weights = np.zeros(20)
    for i in range(50, len(weekly_returns)):
        current_date = weekly_returns.index[i-1]
        next_week = weekly_returns.index[i]

        try:
            weightage = get_stock_weightage(weekly_returns.iloc[:i], current_date, alpha)
            top_stocks = list(weightage.keys())
            new_weights = list(weightage.values())


            next_week_return = weekly_returns.loc[next_week][top_stocks].dot(new_weights)


            transaction_costs = calculate_transaction_costs(prev_weights, new_weights, transaction_cost)
            adjusted_return = next_week_return - transaction_costs
            portfolio_returns.append(adjusted_return)
            weightage_history.append((next_week, weightage))

            prev_weights = new_weights

        except ValueError:
            print(f"Skipping week {next_week} due to insufficient data")

    valid_index = weekly_returns.index[50:50+len(portfolio_returns)]
    return pd.Series(portfolio_returns, index=valid_index), weightage_history


def calculate_drawdown(cumulative_returns):
    drawdown = cumulative_returns / cumulative_returns.cummax() - 1
    max_drawdown = drawdown.min()
    return drawdown, max_drawdown


def sortino_ratio(returns, target=0, periods_per_year=52):
    downside_returns = returns[returns < target]
    downside_deviation = np.sqrt((downside_returns ** 2).mean()) * np.sqrt(periods_per_year)
    annualized_return = returns.mean() * periods_per_year
    return annualized_return / downside_deviation if downside_deviation != 0 else np.nan


def calmar_ratio(annualized_return, max_drawdown):
    return annualized_return / abs(max_drawdown) if max_drawdown != 0 else np.nan


df = pd.read_csv('total_df.csv', index_col='Date', parse_dates=True)


print("Running backtest with transaction costs and exponentially-weighted covariance matrix...")
portfolio_returns, weightage_history = backtest_strategy(df, transaction_cost=0.001, alpha=0.94)


cumulative_returns = (1 + portfolio_returns).cumprod()
total_return = cumulative_returns.iloc[-1]
profit_only_return = total_return - 1
annualized_return = (total_return ** (52/len(cumulative_returns)) - 1)
annualized_volatility = portfolio_returns.std() * np.sqrt(52)
sharpe_ratio = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(52)


drawdown, max_drawdown = calculate_drawdown(cumulative_returns)
calmar = calmar_ratio(annualized_return, max_drawdown)
sortino = sortino_ratio(portfolio_returns)

print("\nBacktest Results with Transaction Costs and Exponentially-Weighted Covariance Matrix:")
print(f"Total Return (including initial capital): {total_return:.4f}")
print(f"Profit-only Return: {profit_only_return:.4%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Calmar Ratio: {calmar:.2f}")


plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
cumulative_returns.plot()
plt.title('Cumulative Portfolio Value (Starting from $1)')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.grid(True)

plt.subplot(2, 1, 2)
drawdown.plot(color='red')
plt.title('Drawdown Over Time')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.grid(True)

plt.tight_layout()
plt.show()


last_date = df.index[-1]
try:
    weightage = get_stock_weightage(calculate_weekly_returns(df), last_date)
    print(f"\nStock weightage for week of {last_date.date()}:")
    for stock, weight in weightage.items():
        print(f"{stock}: {weight:.4f}")
except ValueError as e:
    print(f"Error: {e}")


FileNotFoundError: [Errno 2] No such file or directory: 'total_df.csv'